## test CNN pytorch

In [25]:
#Packages
! pip install requests
! pip install json
! pip install matplotib.pyplot
! pip install time
! pip install numpy 
! pip install yfinance
! pip install pandas
! pip install torch


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement matplotib.pyplot (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for matplotib.pyplot
ERROR: Could not find a version that satisfies the requirement time (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for time

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: py

In [1]:
#Packages
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import yfinance as yf 
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import io
import time

In [68]:
## Test CNN takes 256 x 256 image
class Torch_CNN(nn.Module):
    def __init__(self, embed, num_pred):
        super(Torch_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=embed, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(16, num_pred)  
    
    def forward(self, x): 
        x = torch.nn.functional.leaky_relu(self.conv1(x))  
        x = torch.nn.functional.leaky_relu(self.conv2(x))  
        x = self.gap(x)           
        x = x.view(x.size(0), -1)
        x = self.fc1(x)            
        return x

## Test FNN takes 4 ts values and CNN output
class Torch_FFN(nn.Module):
    def __init__(self, embed, hidden, num_pred):
        super(Torch_FFN, self).__init__()  
        self.fc1 = nn.Linear(embed, hidden) 
        self.fc2 = nn.Linear(hidden, num_pred)  
    
    def forward(self, x):  
        x = torch.nn.functional.leaky_relu(self.fc1(x))
        x = self.fc2(x)            
        return x

# test Ens 
class Torch_Ens(nn.Module):
    def __init__(self, embed_img, embed_ts, hidden_dim, num_pred):
        super().__init__()
        self.cnn = Torch_CNN(embed_img, hidden_dim)  
        self.fnn = Torch_FFN(hidden_dim + embed_ts, hidden_dim, num_pred)
    
    def forward(self, x_img, y_ts):
        x = self.cnn(x_img)
        y_ts = y_ts.view(y_ts.size(0), -1)  
        combined = torch.cat([x, y_ts], dim=1)
        x = self.fnn(combined)
        return x

In [69]:
#Test
## El Model
model = Torch_Ens(
    embed_img=3, #Red / Green / Blue 
    embed_ts=4,      
    hidden_dim=256,  
    num_pred=1        
)

batch_size = 12
x_img = torch.randn(batch_size, 3, 256, 256)  
x_ts = torch.randn(batch_size, 4)             

print(f"Image shape: {x_img.shape}")
print(f"Time series shape: {x_ts.shape}")

output = model(x_img, x_ts)
#print(f"out shape: {output.shape}")  



Image shape: torch.Size([12, 3, 256, 256])
Time series shape: torch.Size([12, 4])


In [ ]:
## crée un dataset on importe d'abord nos données
from torch.utils.data import Dataset, DataLoader

Tickers = ["ZC=F", "ZM=F", "GF=F", "LE=F"] #Corn / Soybean / Feeder cattle / live cattle
data_price = []
for ticker in Tickers:
    data = yf.download(ticker,"2025-02-01", progress = False, auto_adjust=False, interval="1mo") #1year of data
    data_price.append(data['Adj Close'].pct_change().dropna())
data_price

df_close = pd.concat(data_price, axis=1)
df_close.columns = ["Corn","Soy","Cow_food","Cows"] 

df_close.head()

def Get_image_data(df):
    dates =  df.index.strftime('%Y-%m-%d').tolist() #On change le format


    images_dict={}
    for i, date in enumerate(dates):
        if i > 0:
            time.sleep(2)  #On est gentil avec l'API de la Nasa
        params['TIME'] = date 
        print(f"{params['TIME']}") 
        try:
            API_endpoint = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi"
            response = requests.get(API_endpoint, params=params, timeout=30)
            
            if response.status_code == 200:  # Success
                if response.content.startswith(b'\x89PNG'):  
                    img = mpimg.imread(io.BytesIO(response.content))
                    images_dict[date] = img
                else:
                    print(f"Response not PNG for {date}") 
                    print(response.text[:200]) 
                    
            else:
                print(f"Error {response.status_code} for {date}")  
    
        except Exception as e:  
            print(f"Erreur pour {date}: {str(e)}")  
    return images_dict

images_dict = Get_image_data(df_close)

In [70]:
##Ensuite on crée un dataset 
class full_dataset(Dataset):
        def __init__(self, images_dict, df, target_col='Corn'):
                self.dates = df.index.strftime('%Y-%m-%d').tolist()
                self.images = []
                self.ts = []
                self.target = []

                for i, date in enumerate(self.dates):
                        if date not in images_dict:  # Si image manque
                                continue
                        img = images_dict[date]
                        if img.ndim == 3 and img.shape[2] == 4: #IE si aps RGP+
                                img = img[ :, :, :3] #On select que RGB
                        img = img.transpose(2, 0, 1) # pour faire plz a torch 
                        
                        ts = df.loc[date].values #On sort que les valeurs 

                        #on sort les valeurs du corn en T+1 
                        if i < len(self.dates)-1:
                                t_pro = self.dates[i+1]
                                target = df.loc[t_pro,target_col]
                        
                        #On met a jours
                        self.images.append(img) 
                        self.ts.append(ts)
                        self.target.append(target)
                
                #Numpy car torch compatible
                self.images = np.array(self.images)
                self.ts = np.array(self.ts)
                self.target = np.array(self.target)

        def __len__(self):
                return len(self.images) - 1 
        
        def __getitem__(self, idx):
                if idx >= len(self.dates) - 1:
                        idx = len(self.dates) -2 #pour le dernier t + n
                image = torch.from_numpy(self.images[idx])
                ts = torch.from_numpy(self.ts[idx])
                target = torch.tensor(self.target[idx], dtype=torch.float32)
                return image, ts, target
        

dataset = full_dataset(images_dict, df_close,'Corn')

: 

In [ ]:
## On entraine 
model = Torch_Ens(
    embed_img=3, #Red / Green / Blue 
    embed_ts=4,      
    hidden_dim=256,  
    num_pred=1        
)

model.fnn.fc2.bias.data.fill_(0.0) # On initialise le bias a 0

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

split_idx = int(0.8 * len(dataset))  # 80 train, 20% test
train_dataset = torch.utils.data.Subset(dataset, range(split_idx))
val_dataset = torch.utils.data.Subset(dataset, range(split_idx, len(dataset)))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

model.train()
for epoch in range(20):

    model.train()
    total_loss = 0
    for image, ts, target in train_loader:
        image = image.float()
        ts = ts.float()
        target = target.float().unsqueeze(1)

        optimizer.zero_grad()
        output = model(image, ts)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()


    model.eval()
    val_loss = 0
    with torch.no_grad():
        for image, ts, target in val_loader:
            image = image.float()
            ts = ts.float()
            target = target.float().unsqueeze(1)
            output = model(image, ts)
            val_loss += criterion(output, target).item()

    print(f"Epoch {epoch}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")


In [37]:
all_predictions = []
all_targets = []

with torch.no_grad():
    for images, time_series, targets in val_loader:
        predictions = model((images), time_series.float())
        all_predictions.append(predictions)
        all_targets.append(targets.unsqueeze(1))

all_predictions = torch.cat(all_predictions, dim=0)
all_targets = torch.cat(all_targets, dim=0)

print(f"Actual {all_targets[:20]}")
print(f"Predicted {all_predictions[:20]}")

output = model(images.float(), time_series.float())
loss = criterion(output, target.float())
loss.backward()

for name, param in model.named_parameters():
    print(name, param.grad)


Actual tensor([[ 0.0118],
        [-0.0025],
        [ 0.0041],
        [-0.0156],
        [ 0.0082],
        [-0.0153],
        [ 0.0129],
        [ 0.0066],
        [ 0.0056],
        [ 0.0116],
        [-0.0090],
        [ 0.0010],
        [-0.0136],
        [-0.0178],
        [-0.0057],
        [-0.0031],
        [-0.0282],
        [-0.0242],
        [-0.0292],
        [-0.0097]])
Predicted tensor([[ 1.6201e-03],
        [ 1.7040e-03],
        [ 2.2236e-03],
        [ 1.6691e-03],
        [ 2.0539e-03],
        [ 2.0400e-03],
        [ 2.4934e-03],
        [ 2.7171e-03],
        [ 3.3877e-03],
        [ 1.8643e-03],
        [ 1.9400e-03],
        [ 3.8612e-03],
        [ 3.1201e-03],
        [ 2.8275e-03],
        [ 1.7774e-03],
        [ 7.8745e-04],
        [-3.9002e-04],
        [ 6.4855e-04],
        [-1.6209e-05],
        [ 1.3158e-04]])
cnn.conv1.weight tensor([[[[-1.0346e-06, -6.3865e-07, -6.8782e-07],
          [-1.0250e-06, -6.3457e-07, -6.6736e-07],
          [-9.9924e-07